In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.01
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-3
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 1000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.16666667 0.14285714 0.16666667 0.16666667
  0.         0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.16666667 0.14285714 0.16666667 0.
  0.         0.16666667 0.16666667 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.16666667 0.16666667 0.         0.         0.         0.
  0.16666667 0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.16666667]
 [0.14285714 0.14285714 0.         0.         0.         0.14285714
  0.14285714 0.         0.14285714 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.14285714]
 [0.16666667 0.16666667 0.         0.         0.         0.16666667
  0.         0.         0.         0.         0.125      0.
  0.         0.         0.         0

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2, 2])
W2 = np.array([-2, 2])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.2601, grad_fn=<MseLossBackward0>) tensor([ 2.0511, -4.2286]) tensor([-0.1712, -0.5467], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.3593, grad_fn=<MseLossBackward0>) tensor([ 2.9384, -3.7554]) tensor([-0.1918, -0.5044], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(8.2871, grad_fn=<MseLossBackward0>) tensor([ 3.9328, -3.8829]) tensor([-0.2211, -0.4668], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.2735, grad_fn=<MseLossBackward0>) tensor([ 2.6970, -4.0591]) tensor([-0.2605, -0.4280], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(6.3769, grad_fn=<MseLossBackward0>) tensor([ 1.9200, -3.9647]) tensor([-0.2874, -0.3874], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(6.4627, grad_fn=<MseLossBackward0>) tensor([ 2.7510, -3.5211]) tensor([-0.3066, -0.3478], grad_fn=<CatBackward0>)
torch.Size([50, 1])
to

16 tensor(0.9110, grad_fn=<MseLossBackward0>) tensor([ 0.7147, -1.5064]) tensor([-1.3591,  1.0946], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
16 tensor(0.9207, grad_fn=<MseLossBackward0>) tensor([ 1.0261, -1.3379]) tensor([-1.3662,  1.1097], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
16 tensor(1.0349, grad_fn=<MseLossBackward0>) tensor([ 1.3761, -1.3820]) tensor([-1.3765,  1.1230], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
16 tensor(0.9110, grad_fn=<MseLossBackward0>) tensor([ 0.9416, -1.4457]) tensor([-1.3903,  1.1369], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.8002, grad_fn=<MseLossBackward0>) tensor([ 0.6692, -1.4122]) tensor([-1.3997,  1.1513], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.8086, grad_fn=<MseLossBackward0>) tensor([ 0.9609, -1.2542]) tensor([-1.4064,  1.1654], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.9088

torch.Size([50, 1])
torch.Size([50, 1])
33 tensor(0.1002, grad_fn=<MseLossBackward0>) tensor([ 0.3093, -0.4816]) tensor([-1.7992,  1.7129], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0881, grad_fn=<MseLossBackward0>) tensor([ 0.2195, -0.4705]) tensor([-1.8023,  1.7177], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0889, grad_fn=<MseLossBackward0>) tensor([ 0.3157, -0.4178]) tensor([-1.8045,  1.7224], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0997, grad_fn=<MseLossBackward0>) tensor([ 0.4240, -0.4313]) tensor([-1.8076,  1.7266], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0880, grad_fn=<MseLossBackward0>) tensor([ 0.2897, -0.4514]) tensor([-1.8119,  1.7309], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
35 tensor(0.0774, grad_fn=<MseLossBackward0>) tensor([ 0.2056, -0.4410]) tensor([-1.8148,  1.7354], grad_fn=<CatBackward0>)
torch.Size([50, 

torch.Size([50, 1])
torch.Size([50, 1])
49 tensor(0.0126, grad_fn=<MseLossBackward0>) tensor([ 0.1088, -0.1709]) tensor([-1.9292,  1.8982], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
50 tensor(0.0111, grad_fn=<MseLossBackward0>) tensor([ 0.0771, -0.1670]) tensor([-1.9303,  1.8999], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
50 tensor(0.0111, grad_fn=<MseLossBackward0>) tensor([ 0.1111, -0.1483]) tensor([-1.9311,  1.9016], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
50 tensor(0.0125, grad_fn=<MseLossBackward0>) tensor([ 0.1493, -0.1529]) tensor([-1.9322,  1.9031], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
50 tensor(0.0110, grad_fn=<MseLossBackward0>) tensor([ 0.1019, -0.1602]) tensor([-1.9337,  1.9046], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
51 tensor(0.0097, grad_fn=<MseLossBackward0>) tensor([ 0.0723, -0.1565]) tensor([-1.9347,  1.9062], grad_fn=<CatBackward0>)
torch.Size([50, 

66 tensor(0.0014, grad_fn=<MseLossBackward0>) tensor([ 0.0272, -0.0593]) tensor([-1.9754,  1.9645], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
66 tensor(0.0014, grad_fn=<MseLossBackward0>) tensor([ 0.0392, -0.0526]) tensor([-1.9757,  1.9651], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
66 tensor(0.0016, grad_fn=<MseLossBackward0>) tensor([ 0.0526, -0.0541]) tensor([-1.9761,  1.9657], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
66 tensor(0.0014, grad_fn=<MseLossBackward0>) tensor([ 0.0359, -0.0568]) tensor([-1.9766,  1.9662], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
67 tensor(0.0012, grad_fn=<MseLossBackward0>) tensor([ 0.0254, -0.0556]) tensor([-1.9770,  1.9668], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
67 tensor(0.0012, grad_fn=<MseLossBackward0>) tensor([ 0.0367, -0.0493]) tensor([-1.9772,  1.9673], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
67 tensor(0.0014

80 tensor(0.0002, grad_fn=<MseLossBackward0>) tensor([ 0.0158, -0.0212]) tensor([-1.9902,  1.9859], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
80 tensor(0.0003, grad_fn=<MseLossBackward0>) tensor([ 0.0212, -0.0217]) tensor([-1.9904,  1.9862], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
80 tensor(0.0002, grad_fn=<MseLossBackward0>) tensor([ 0.0144, -0.0229]) tensor([-1.9906,  1.9864], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
81 tensor(0.0002, grad_fn=<MseLossBackward0>) tensor([ 0.0102, -0.0225]) tensor([-1.9907,  1.9866], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
81 tensor(0.0002, grad_fn=<MseLossBackward0>) tensor([ 0.0148, -0.0199]) tensor([-1.9908,  1.9868], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
81 tensor(0.0002, grad_fn=<MseLossBackward0>) tensor([ 0.0198, -0.0204]) tensor([-1.9910,  1.9870], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
81 tensor(0.0002

93 tensor(4.2070e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0047, -0.0104]) tensor([-1.9958,  1.9939], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
93 tensor(4.2091e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0068, -0.0091]) tensor([-1.9958,  1.9940], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
93 tensor(4.6065e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0091, -0.0093]) tensor([-1.9959,  1.9940], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
93 tensor(4.1567e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0062, -0.0099]) tensor([-1.9960,  1.9941], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(3.6981e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0044, -0.0097]) tensor([-1.9960,  1.9942], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(3.6985e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0064, -0.0086]) tensor([-1.9961,  1.9943], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([

In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9973,  1.9961], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, 0, tensor([[-0.1204,  0.0000],
        [ 0.0000, -0.1204]]), tensor([[2.4567, 0.0000],
        [0.0000, 2.4567]]), 0, tensor([[1.6217, 0.0000],
        [0.0000, 1.6217]]), tensor([[-0.5137,  0.0000],
        [ 0.0000, -0.5137]]), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(parameters_to_vector(model.parameters()))).detach()

In [18]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.2392,  0.4220])


tensor([-0.4784,  0.8441], grad_fn=<CatBackward0>)

In [19]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []


for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros_like(weights)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[k][i] - projected_weights[i][k])
            #print(torch.outer(mat_vec_sum, weights))

            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * mat_vec_sum)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    total_rel_error = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/1000 [00:00<02:22,  7.02it/s]

Training_loss 8.64441,   Relative Error 5.49577


  0%|          | 2/1000 [00:00<02:10,  7.62it/s]

Training_loss 8.61836,   Relative Error 5.52751


  0%|          | 3/1000 [00:00<02:05,  7.92it/s]

...

Training_loss 0.36029,   Relative Error 0.30832
Training_loss 0.35916,   Relative Error 0.30768
Training_loss 0.35870,   Relative Error 0.30743
Training_loss 0.35815,   Relative Error 0.30712


100%|█████████▉| 999/1000 [01:14<00:00, 16.71it/s]

Training_loss 0.35670,   Relative Error 0.30632
Training_loss 0.35610,   Relative Error 0.30598
Training_loss 0.35417,   Relative Error 0.30491
Training_loss 0.35301,   Relative Error 0.30426


100%|██████████| 1000/1000 [01:14<00:00, 13.41it/s]

Training_loss 0.35247,   Relative Error 0.30396


In [20]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-1.6075,  1.4674], grad_fn=<CatBackward0>)

In [21]:
for j in G.neighbors(0):
    print(j)

3
4
6
7


In [22]:
parameters_to_vector(models[0].parameters())

tensor([1.6508, 1.7437], grad_fn=<CatBackward0>)

In [23]:
projection_list[0]

[0,
 0,
 0,
 tensor([[-0.1177,  0.0020],
         [ 0.0027, -0.1184]]),
 tensor([[2.4594e+00, 1.9614e-03],
         [2.7227e-03, 2.4586e+00]]),
 0,
 tensor([[1.6244, 0.0020],
         [0.0027, 1.6236]]),
 tensor([[-0.5110,  0.0020],
         [ 0.0027, -0.5118]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [24]:
projected_weights[0]

[0,
 0,
 0,
 tensor([3.7269, 3.7729]),
 tensor([1.6052, 1.6399]),
 0,
 tensor([1.5965, 1.4719]),
 tensor([2.2503, 2.1960]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [25]:
test_loss = np.asarray(test_loss)
total_rel_error = np.asarray(total_rel_error)

In [26]:
np.save( 'training_loss_sheave_fml', test_loss)
np.save('relative_error_sheave_fml', total_rel_error)